##### MAIN CODE

In [ ]:
#!/usr/bin/env python3
import os
import logging
from io import StringIO

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from scipy.optimize import curve_fit
import emcee

# ------------------ USER SETTINGS ------------------
#YEAR = 2015
#OGLE = 'ogle4' if YEAR > 2009 else 'ogle3'
#ZERO_COUNT = 4 if YEAR > 2009 else 3

# Event loop (inclusive)
#N_i = 3
#N_f = 3

# MCMC settings
RNG_SEED = 123
N_WALKERS = 64
N_STEPS = 10000
N_BURN = 500
THIN = 2

# Output
OUT_DIR = "data"
#OUT_CSV = f"{N_i}-{N_f}-{YEAR}.csv"

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


# ------------------ MODEL ------------------
def pspl_blend(t, Fs, Fb, u0, t0, tE):
    """Blended PSPL model: total flux = Fs*A(u) + Fb."""
    u = np.sqrt(u0**2 + ((t - t0) / tE)**2)
    A = (u**2 + 2.0) / (u * np.sqrt(u**2 + 4.0))
    return Fs * A + Fb


# ------------------ I/O HELPERS ------------------
def load_ogle_df(year, event, ogle, zero_count=4, timeout=30):
    idx = str(event).zfill(zero_count)
    url = f"https://www.astrouw.edu.pl/ogle/{ogle}/ews/{year}/blg-{idx}/phot.dat"
    logging.info(f"Downloading photometry: {url}")
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    df = pd.read_csv(
    StringIO(r.text),
    sep=r"\s+",
    comment="#",
    header=None,
    on_bad_lines="skip"   # <-- skip malformed rows
    )
    df = df.iloc[:, :3]
    df.columns = ["Time", "I_mag", "Mag_err"]
    return df


def fetch_ogle_guesses(year, event, ogle, zero_count=4, timeout=30):
    """
    Fetch initial guesses from the OGLE HTML page.
    Expected mapping (row index 1 of the 'guesses' table):
      t0 = dfs[2].iloc[1, 0]
      tE = dfs[2].iloc[1, 1]
      u0 = dfs[2].iloc[1, 2]
      fs = dfs[2].iloc[1, 5]
      I0 = dfs[2].iloc[1, 6]
    Falls back to dfs[0] if structure differs.
    """
    idx = str(event).zfill(zero_count)
    url_ogle = f"https://ogle.astrouw.edu.pl/{ogle}/ews/{year}/blg-{idx}.html"
    logging.info(f"Fetching OGLE guesses: {url_ogle}")
    try:
        r = requests.get(url_ogle, timeout=timeout)
        r.raise_for_status()
    except Exception as e:
        logging.warning(f"Failed OGLE page {url_ogle}: {e}")
        return None

    try:
        dfs = pd.read_html(r.text)
        guess_table = dfs[2] if len(dfs) >= 3 else dfs[0]
    except Exception as e:
        logging.warning(f"Unable to parse OGLE HTML tables: {e}")
        return None

    try:
        T_max_OGLE          = float(dfs[2][1][0])
        T_max_erorr_OGLE    = float(dfs[2][3][0])
        T_E_OGLE            = float(dfs[2][1][1])
        T_E_error_OGLE      = float(dfs[2][3][1])
        u_0_OGLE            = float(dfs[2][1][2])
        u_0_error_OGLE      = float(dfs[2][3][2])
        f_s_OGLE            = float(dfs[2][1][5])
        f_s_error_OGLE      = float(dfs[2][3][5])
        I_0_OGLE            = float(dfs[2][1][7])
        I_s_error_OGLE      = float(dfs[2][3][7])

        t0 = T_max_OGLE
        tE = T_E_OGLE
        u0 = u_0_OGLE
        fs = f_s_OGLE
        I0 = I_0_OGLE
        return {"t0": t0, "tE": tE, "u0": u0, "fs": fs, "I0": I0}
    except Exception as e:
        logging.warning(f"Parsed HTML but couldn't extract expected cells: {e}")
        return None


# ------------------ FITTING PIPELINE FOR ONE EVENT ------------------
def fit_event(year, event, ogle, zero_count, out_dir):
    rng = np.random.default_rng(RNG_SEED + int(event))  # small variation per event

    # Load full photometry (we'll keep full arrays for plotting)
    df = load_ogle_df(year, event, ogle, zero_count)
    t_full = df['Time'].to_numpy(float)
    F_raw = 10 ** (-0.4 * df['I_mag'].to_numpy(float))
    F_err_raw = F_raw * (np.log(10) * 0.4) * df['Mag_err'].to_numpy(float)

    # Normalize by median baseline
    norm = np.median(F_raw)
    flux_full = F_raw / norm
    flux_err_full = np.maximum(F_err_raw / norm, 1e-6)

    # OGLE HTML guesses
    og = fetch_ogle_guesses(year, event, ogle, zero_count)
    if og is None:
        # Data-based fallback
        baseline = float(np.median(flux_full))
        peak = float(np.max(flux_full))
        t0_guess = float(t_full[np.argmax(flux_full)])
        Fs_guess = max(peak - baseline, 1e-6)
        Fb_guess = max(baseline - Fs_guess, 1e-6)

        def u_from_A(A):
            A = max(A, 1.0001)
            return np.sqrt(max(2.0 * (A / np.sqrt(A*A - 1.0)) - 2.0, 1e-8))
        Amax = peak / max(baseline, 1e-12)
        u0_guess = float(np.clip(u_from_A(Amax), 1e-4, 0.9))
        tE_guess = float(max((t_full.max() - t_full.min()) / 6.0, 0.5))

        ogle_init_for_table = {"Fs": np.nan, "Fb": np.nan, "u0": np.nan, "t0": np.nan, "tE": np.nan}
    else:
        # Convert OGLE (fs, I0) to our scale Fs/Fb
        t0_guess = float(og["t0"])
        tE_guess = float(max(og["tE"], 1e-4))
        u0_guess = float(np.clip(og["u0"], 1e-6, 0.9999))
        fs_ogle = float(np.clip(og["fs"], 1e-8, 1-1e-8))
        I0_guess = float(og["I0"])
        F0_abs = 10 ** (-0.4 * I0_guess)
        F0_norm = F0_abs / norm
        Fs_guess = fs_ogle * F0_norm
        Fb_guess = (1.0 - fs_ogle) * F0_norm

        ogle_init_for_table = {
            "Fs": float(Fs_guess), "Fb": float(Fb_guess),
            "u0": float(u0_guess), "t0": float(t0_guess), "tE": float(tE_guess)
        }

    # Limit fitting data to [t0 - 3 tE, t0 + 3 tE]
    tmin_fit = t0_guess - 3.0 * tE_guess
    tmax_fit = t0_guess + 3.0 * tE_guess
    mask_fit = (t_full >= tmin_fit) & (t_full <= tmax_fit)
    if not np.any(mask_fit):
        raise RuntimeError("No data within the requested fit window.")

    t = t_full[mask_fit]
    flux = flux[mask_fit] if 'flux' in locals() else flux_full[mask_fit]  # keep structure
    flux = flux_full[mask_fit]
    flux_err = flux_err_full[mask_fit]
    span_fit = float(t.max() - t.min())

    # -------- curve_fit (with safe bounds + clamped p0) --------
    p0_cf = np.array([Fs_guess, Fb_guess, u0_guess, t0_guess, tE_guess], dtype=float)
    lower = np.array([1e-12, 0.0,   1e-6,   t.min(), 1e-4], dtype=float)
    upper = np.array([1e6,   1e6,  0.9999,  t.max(), max(span_fit*10.0, 10.0)], dtype=float)
    eps = 1e-9
    p0_cf = np.minimum(np.maximum(p0_cf, lower + eps), upper - eps)

    popt, pcov = curve_fit(
        pspl_blend, t, flux, sigma=flux_err,
        p0=p0_cf, bounds=(lower, upper),
        absolute_sigma=True, maxfev=20000
    )
    Fs_cf, Fb_cf, u0_cf, t0_cf, tE_cf = popt
    perr = np.sqrt(np.diag(pcov))
    err_Fs, err_Fb, err_u0, err_t0, err_tE = perr

    # ---- NEW: curve_fit f_s and its 1-sigma uncertainty (linear propagation) ----
    f_s_cf = Fs_cf / (Fs_cf + Fb_cf)
    # derivatives of f = Fs/(Fs+Fb)
    denom2 = (Fs_cf + Fb_cf)**2
    df_dFs = Fb_cf / denom2
    df_dFb = -Fs_cf / denom2
    var_fs = (df_dFs**2) * pcov[0, 0] + (df_dFb**2) * pcov[1, 1] + 2.0 * df_dFs * df_dFb * pcov[0, 1]
    f_s_cf_err = float(np.sqrt(max(var_fs, 0.0)))

    # -------- MCMC (on the same limited window) --------
    # Seeds from the limited window (data-driven)
    baseline = float(np.median(flux))
    peak = float(np.max(flux))
    t0_data = float(t[np.argmax(flux)])
    Fs_data = max(peak - baseline, 1e-6)
    Fb_data = max(baseline - Fs_data, 1e-6)
    def u_from_A(A):
        A = max(A, 1.0001)
        return np.sqrt(max(2.0 * (A / np.sqrt(A*A - 1.0)) - 2.0, 1e-8))
    Amax = peak / max(baseline, 1e-12)
    u0_data = float(np.clip(u_from_A(Amax), 1e-4, 0.9))
    tE_data = float(max(span_fit / 6.0, 0.5))

    def log_prior(theta):
        Fs, Fb, u0, t0p, tEp = theta
        if Fs <= 0 or Fb < 0: return -np.inf
        if not (1e-6 < u0 < 1.0): return -np.inf
        if not (t.min() <= t0p <= t.max()): return -np.inf
        if tEp <= 1e-4: return -np.inf
        # gentle log-normal around seeds
        lp = 0.0
        for val, ref in [(Fs, Fs_data), (Fb + 1e-12, Fb_data + 1e-12), (tEp, tE_data)]:
            lp -= 0.5 * ((np.log(val) - np.log(ref)) / 2.0)**2
        return lp

    def log_likelihood(theta):
        model = pspl_blend(t, *theta)
        var = flux_err**2
        return -0.5 * np.sum(np.log(2*np.pi*var) + (flux - model)**2 / var)

    def log_prob(theta):
        lp = log_prior(theta)
        if not np.isfinite(lp): return -np.inf
        return lp + log_likelihood(theta)

    p0_mcmc = np.array([Fs_data, Fb_data, u0_data, t0_data, tE_data], dtype=float)
    p_start = p0_mcmc * (1.0 + 3e-2 * rng.normal(size=(N_WALKERS, p0_mcmc.size)))
    p_start[:, 0] = np.maximum(p_start[:, 0], 1e-6)
    p_start[:, 1] = np.maximum(p_start[:, 1], 0.0)
    p_start[:, 2] = np.clip(p_start[:, 2], 1e-5, 0.9)
    p_start[:, 3] = np.clip(p_start[:, 3], t.min(), t.max())
    p_start[:, 4] = np.maximum(p_start[:, 4], 1e-3)

    sampler = emcee.EnsembleSampler(N_WALKERS, p0_mcmc.size, log_prob)
    sampler.run_mcmc(p_start, N_STEPS, progress=False)
    samples = sampler.get_chain(discard=N_BURN, thin=THIN, flat=True)

    names = ['Fs', 'Fb', 'u0', 't0', 'tE']
    med = np.median(samples, axis=0)
    p16 = np.percentile(samples, 16, axis=0)
    p84 = np.percentile(samples, 84, axis=0)
    Fs_m, Fb_m, u0_m, t0_m, tE_m = med

    # ---- NEW: MCMC f_s and its 16/50/84 percentiles ----
    f_s_samp = samples[:, 0] / (samples[:, 0] + samples[:, 1])
    f_s_med = float(np.median(f_s_samp))
    f_s_lo = float(np.percentile(f_s_samp, 16))
    f_s_hi = float(np.percentile(f_s_samp, 84))

    # ------------------ PLOTS & SAVES ------------------
    os.makedirs(out_dir, exist_ok=True)
    idx_str = str(event).zfill(zero_count)

    # Build model curves across full x-range
    tt_full = np.linspace(t_full.min(), t_full.max(), 3000)
    curve_cf_full = pspl_blend(tt_full, Fs_cf, Fb_cf, u0_cf, t0_cf, tE_cf)

    # MCMC model credible band (68%) around the curve
    nsamp_plot = min(400, samples.shape[0])
    sel = rng.choice(samples.shape[0], size=nsamp_plot, replace=False)
    preds = np.array([pspl_blend(tt_full, *samples[i]) for i in sel])
    pred_low = np.percentile(preds, 16, axis=0)
    pred_med = np.percentile(preds, 50, axis=0)
    pred_high = np.percentile(preds, 84, axis=0)

    curve_mc_full = pred_med  # median curve
    res_cf_full = flux_full - pspl_blend(t_full, Fs_cf, Fb_cf, u0_cf, t0_cf, tE_cf)
    res_mc_full = flux_full - pspl_blend(t_full, Fs_m,  Fb_m,  u0_m,  t0_m,  tE_m)

    # 3-panel figure
    fig = plt.figure(figsize=(10, 10))
    ax0 = plt.subplot2grid((7,1), (0,0), rowspan=3)  # 3/7 of height
    ax1 = plt.subplot2grid((7,1), (3,0), rowspan=3)  # 3/7 of height
    ax2 = plt.subplot2grid((7,1), (6,0), rowspan=2)  # 1/7 of height
    ax = [ax0, ax1, ax2]
    
    plt.subplots_adjust(hspace=0.12)
    errkw = dict(elinewidth=1, capsize=2)

    ax[0].errorbar(t_full, flux_full, yerr=flux_err_full,
                            fmt='.', ms=3, **errkw, label='data',color = 'black' ,  alpha = 0.4)
    ax[0].plot(tt_full, curve_cf_full, lw=2, label='curve_fit ', color = 'red')
    ax[0].plot(tt_full, curve_mc_full, lw=2, label='MCMC median', color='green', alpha = 0.8)
    ax[0].axvline(t0_m, ls='--', lw=1, label='MCMC peak (t0)',color= 'blue')
    ax[0].axvspan(tmin_fit, tmax_fit, color='grey', alpha=0.15, label='fit window')
    ax[0].set_ylabel('Normalized flux')
    ax[0].set_title(f'OGLE-BLG-{year}-{event} PSPL fit')
    ax[0].legend(loc='best')

    # --- MCMC model prediction band (68% CI) ---
    rng = np.random.default_rng(RNG_SEED)
    idxs = rng.choice(samples.shape[0], size=300, replace=False)

    preds = np.array([
        pspl_blend(tt_full, *samples[i])
        for i in idxs
    ])
    pred_low = np.percentile(preds, 16, axis=0)
    pred_med = np.percentile(preds, 50, axis=0)
    pred_high = np.percentile(preds, 84, axis=0)

    # --- your plot ---
    ax[1].errorbar(t_full, flux_full, yerr=flux_err_full,
                    fmt='.', ms=3, **errkw, color = 'black' ,  alpha = 0.4)
    ax[1].plot(tt_full, curve_cf_full, lw=2, label='curve_fit', color = 'red')
    ax[1].plot(tt_full, pred_med, lw=2, label='MCMC median', color='green', alpha = 0.8)
    ax[1].fill_between(
        tt_full, pred_low, pred_high,
        color='green', alpha=0.3, label='MCMC 68% CI'
    )
    ax[1].axvline(t0_m, ls='--', lw=1, color='blue')
    ax[1].set_xlim(t0_m - 3*tE_m, t0_m + 3*tE_m)
    ax[1].set_ylabel('Normalized flux')

    # Left-top box: MCMC medians ± (16/84%)  ---- ADDED f_s ----
    mcmc_lines = [f"{n} = {m:.6g} (+{(h-m):.3g}/-{(m-l):.3g})"
                for n, m, l, h in zip(names, med, p16, p84)]
    mcmc_lines.append(f"f_s = {f_s_med:.6g} (+{(f_s_hi - f_s_med):.3g}/-{(f_s_med - f_s_lo):.3g})")
    ax[1].text(
        0.02, 0.98, "\n".join(mcmc_lines),
        transform=ax[1].transAxes, va='top', ha='left',
        fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.9)
    )
    ax[1].legend(loc='upper right')

    # ax[2] Residuals (FULL series, but zoom x-range to same as ax[1])
    errkw_res = dict(elinewidth=1, capsize=2, alpha=0.4)
    ax[2]
    ax[2].axhline(0.0, ls='--', lw=1, color='gray')
    ax[2].errorbar(t_full, res_cf_full, yerr=flux_err_full, fmt='.', ms=3,
                    **errkw_res, label='data - curve_fit', color = 'red')
    ax[2].errorbar(t_full, res_mc_full, yerr=flux_err_full, fmt='.', ms=3,
                    **errkw_res, label='data - MCMC' , color = 'green')
    ax[2].set_xlabel('HJD')
    ax[2].set_ylabel('Residual')
    ax[2].set_xlim(t0_m - 3*tE_m, t0_m + 3*tE_m)
    ax[2].axvline(t0_m, ls='--', lw=1, color='blue')
    ax[2].legend(loc='upper left')

    plt.tight_layout()
    #plt.show()
    fit_png = os.path.join(out_dir, f"OGLE{year}-{idx_str}_fit.png")
    fig.savefig(fit_png, dpi=200, bbox_inches="tight")
    plt.close(fig)

    # Corner plot (colored)
    try:
        import corner
        fig_corner = corner.corner(
            samples,
            labels=[r"$F_s$", r"$F_b$", r"$u_0$", r"$t_0$", r"$t_E$"],
            quantiles=[0.16, 0.5, 0.84],
            show_titles=True,
            title_fmt=".3g",
            truths=med,
            plot_datapoints=True,
            plot_density=True,
            fill_contours=True,
            color="C0"
        )
        fig_corner.suptitle(f"OGLE{year}-{idx_str} — MCMC Posterior", y=1.02)
        corner_png = os.path.join(out_dir, f"OGLE{year}-{idx_str}_corner.png")
        fig_corner.savefig(corner_png, dpi=200, bbox_inches="tight")
        plt.close(fig_corner)
    except Exception as e:
        logging.warning(f"corner plot failed: {e}")

    # ------------------ SUMMARY ROW ------------------
    # init f_s if available
    if not np.isnan(ogle_init_for_table["Fs"]) and not np.isnan(ogle_init_for_table["Fb"]):
        init_fs = ogle_init_for_table["Fs"] / (ogle_init_for_table["Fs"] + ogle_init_for_table["Fb"])
    else:
        init_fs = np.nan

    row = {
        "year": year, "event": event, "idx": idx_str,
        # OGLE-seeded init (normalized scale)
        "init_Fs": ogle_init_for_table["Fs"],
        "init_Fb": ogle_init_for_table["Fb"],
        "init_u0": ogle_init_for_table["u0"],
        "init_t0": ogle_init_for_table["t0"],
        "init_tE": ogle_init_for_table["tE"],
        "init_f_s": init_fs,                                # <-- ADDED

        # curve_fit (limited window) + 1σ errors
        "cf_Fs": Fs_cf, "cf_Fs_err": err_Fs,
        "cf_Fb": Fb_cf, "cf_Fb_err": err_Fb,
        "cf_u0": u0_cf, "cf_u0_err": err_u0,
        "cf_t0": t0_cf, "cf_t0_err": err_t0,
        "cf_tE": tE_cf, "cf_tE_err": err_tE,
        "cf_f_s": f_s_cf, "cf_f_s_err": f_s_cf_err,        # <-- ADDED

        # MCMC medians and 16/84 widths
        "mc_Fs": med[0], "mc_Fs_m": med[0]-p16[0], "mc_Fs_p": p84[0]-med[0],
        "mc_Fb": med[1], "mc_Fb_m": med[1]-p16[1], "mc_Fb_p": p84[1]-med[1],
        "mc_u0": med[2], "mc_u0_m": med[2]-p16[2], "mc_u0_p": p84[2]-med[2],
        "mc_t0": med[3], "mc_t0_m": med[3]-p16[3], "mc_t0_p": p84[3]-med[3],
        "mc_tE": med[4], "mc_tE_m": med[4]-p16[4], "mc_tE_p": p84[4]-med[4],
        "mc_f_s": f_s_med,                                  # <-- ADDED
        "mc_f_s_m": f_s_med - f_s_lo,                       # lower error
        "mc_f_s_p": f_s_hi - f_s_med,                       # upper error

        # bookkeeping
        "n_full": int(t_full.size),
        "n_fit": int(t.size),
        "fit_window_min": tmin_fit,
        "fit_window_max": tmax_fit,
        "fit_png": fit_png
    }
    return row


# ------------------ DRIVER ------------------
def main():
    os.makedirs(OUT_DIR, exist_ok=True)
    data_rep = pd.DataFrame()

    season = [ (2002 , 5) , (2002 , 389)   , (2003 , 462 ) , (2004 , 608 ) , (2005 , 597)  ,
               (2006 , 581)   , (2007 , 610 ) , (2008 , 654 ) , (2009 , 159)  ,
               (2011 , 1562)  , (2012 , 1744) , (2013 , 1987) , (2014 , 2049) ,
               (2015 , 2145 ) , (2016 , 1927) , (2017 , 1834) , (2018 , 1803) ,
               (2019 , 1526) ]

    for can in season:
        
        YEAR = can[0]
        N_f = can[1]
        print(YEAR, "started")
        
        OGLE = 'ogle4' if YEAR > 2009 else 'ogle3'
        ZERO_COUNT = 4 if YEAR > 2009 else 3

        data_rep = pd.DataFrame()

        for i in range (1 , N_f+1):
            print('OGLE',YEAR,'BLG',i,"started")
            try:
                row = fit_event(YEAR, i, OGLE, ZERO_COUNT, OUT_DIR)
                data_rep = pd.concat([data_rep, pd.DataFrame([row])], ignore_index=True)
                print(YEAR, "-", i, "done")
            except Exception as e:
                logging.warning(f"Event {i}: failed with error: {e}")
                continue
            print('OGLE',YEAR,'BLG',i,"Done")

        data_rep.to_csv(os.path.join(OUT_DIR, f'OGLE-{YEAR}-{i}.csv'), index=False)

if __name__ == "__main__":
    main()


INFO: Downloading photometry: https://www.astrouw.edu.pl/ogle/ogle3/ews/2002/blg-001/phot.dat


2002 started
OGLE 2002 BLG 1 started


INFO: Fetching OGLE guesses: https://ogle.astrouw.edu.pl/ogle3/ews/2002/blg-001.html
C:\Users\Saeed\AppData\Local\Temp\ipykernel_14856\3667917964.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(r.text)
INFO: Downloading photometry: https://www.astrouw.edu.pl/ogle/ogle3/ews/2002/blg-002/phot.dat


2002 - 1 done
OGLE 2002 BLG 1 Done
OGLE 2002 BLG 2 started


INFO: Fetching OGLE guesses: https://ogle.astrouw.edu.pl/ogle3/ews/2002/blg-002.html
C:\Users\Saeed\AppData\Local\Temp\ipykernel_14856\3667917964.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(r.text)
INFO: Downloading photometry: https://www.astrouw.edu.pl/ogle/ogle3/ews/2002/blg-003/phot.dat


...

OGLE 2003 BLG 353 started


INFO: Fetching OGLE guesses: https://ogle.astrouw.edu.pl/ogle3/ews/2003/blg-353.html
C:\Users\Saeed\AppData\Local\Temp\ipykernel_14856\3667917964.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(r.text)
INFO: Downloading photometry: https://www.astrouw.edu.pl/ogle/ogle3/ews/2003/blg-354/phot.dat


OGLE 2003 BLG 354 started


INFO: Fetching OGLE guesses: https://ogle.astrouw.edu.pl/ogle3/ews/2003/blg-354.html
C:\Users\Saeed\AppData\Local\Temp\ipykernel_14856\3667917964.py:85: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(r.text)


KeyboardInterrupt: 